# Part 2:  Join Examples

We are going to use the following website to practice SQL joins: https://www.w3schools.com/sql/trysql.asp?filename=trysql_asc


## Joins

**people**  

| id       | name           | age | city |
| ------------- |-------------| -----|---|
| 1     | Joe | 5| Durango|
| 2      | Alice     | 55| Albuquerque |
| 3 | Veronica     |   105| Albuquerque |
|4| Matt | 43 | null |

**cities**    

| id       | city       | state | population|
| ------------- |-------------| -----|---|
| 1     | Tijeras| NM |987|
| 2      | Albuquerque | NM | 8776|
| 3 | Santa Fe    |  NM|907|
|4| Durango| CO| 9|


## Inner Join
Inner join returns data where there are matching records in both tables. There must be some matching data in the two (or more) tables that are being joined. In the above example, we can perform an inner join on the people and cities tables because they have the city column in common. 
An inner join on cities will only return the cities they both have in common. 

### Inner Join Syntax 
`SELECT * 

FROM people 

INNER JOIN cities ON people.city = cities.city`


### Inner Join Example

| id       | name           | age | city | state | population
| ------------- |-------------| -----|---| ------| -----
| 1     | Joe | 5| Durango| CO | 9
| 2      | Alice     | 55| Albuquerque | NM | 8776
| 3 | Veronica     |   105| Albuquerque | NM | 8776

## Outer Join
An outer join can be made on the left or right table. 
An outer join from the left table returns ALL rows from the left table and nulls for columns on the right table if no entries match. 

### Outer Join Syntax
`SELECT * FROM people LEFT JOIN cities ON people.city = cities.city`

### Outer Join Example
| id       | name           | age | city | state | population
| ------------- |-------------| -----|---| ------| -----
| 1     | Joe | 5| Durango| CO | 9
| 2      | Alice     | 55| Albuquerque | NM | 8776
| 3 | Veronica     |   105| Albuquerque | NM | 8776
| 4 | Matt | 43 | null | null | null

[Example](https://www.w3schools.com/sql/sql_join_inner.asp)

##Inner Join with BigQuery 

Inner Joins return data where there are matching records in both tables.  

[Join documentation on BigQuery](https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax#join_types)

[Kaggle Inner Joins](https://www.kaggle.com/dansbecker/joining-data)  
[Kaggle Outter Joins](https://www.kaggle.com/alexisbcook/joins-and-unions)

Inner and Outer Joins Venn Diagram https://realpython.com/pandas-merge-join-and-concat/ about halfway down

In [2]:
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()

In [3]:
project_id = 'ds-7-lecture-demo'

# Create client object
client = bigquery.Client(project=project_id)

In [4]:
dataset_ref = client.dataset("github_repos", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "licenses" table
licenses_ref = dataset_ref.table("licenses")

# API request - fetch the table
licenses_table = client.get_table(licenses_ref)

# Preview the first five lines of the "licenses" table
client.list_rows(licenses_table, max_results=5).to_dataframe()

,repo_name,license
0,zoffixznet/perl6-App-Nopaste,artistic-2.0
1,vikt0rs/devstack-plugin-zmq,artistic-2.0
2,LegitTalon/nicolas,artistic-2.0
3,Arabidopsis-Information-Portal/Intern-Hello-World,artistic-2.0
4,andrewdonkin/falcon,artistic-2.0


In [5]:
# Construct a reference to the "sample_files" table
files_ref = dataset_ref.table("sample_files")

# API request - fetch the table
files_table = client.get_table(files_ref)

# Preview the first five lines of the "sample_files" table
client.list_rows(files_table, max_results=5).to_dataframe()


,repo_name,ref,path,mode,id,symlink_target
0,git/git,refs/heads/master,RelNotes,40960,62615ffa4e97803da96aefbc798ab50f949a8db7,Documentation/RelNotes/2.10.0.txt
1,np/ling,refs/heads/master,tests/success/plug_compose.t/plug_compose.ll,40960,0c1605e4b447158085656487dc477f7670c4bac1,../../../fixtures/all/plug_compose.ll
2,np/ling,refs/heads/master,fixtures/strict-par-success/parallel_assoc_lef...,40960,b59bff84ec03d12fabd3b51a27ed7e39a180097e,../all/parallel_assoc_left.ll
3,np/ling,refs/heads/master,fixtures/sequence/parallel_assoc_2tensor2_left.ll,40960,f29523e3fb65702d99478e429eac6f801f32152b,../all/parallel_assoc_2tensor2_left.ll
4,np/ling,refs/heads/master,fixtures/success/my_dual.ll,40960,38a3af095088f90dfc956cb990e893909c3ab286,../all/my_dual.ll


In [6]:
# Add safe config settings
ONE_MB = 1000*1000
TWO_GB = 2*1000*ONE_MB


In [7]:
query = """
        SELECT files.repo_name, licenses.license
        FROM `bigquery-public-data.github_repos.sample_files` AS files
        INNER JOIN `bigquery-public-data.github_repos.licenses` AS licenses
          ON files.repo_name = licenses.repo_name
        LIMIT 5
        """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=TWO_GB)
licenses = client.query(query, job_config=safe_config).to_dataframe()
licenses.head()

,repo_name,license
0,ukhas/dl-fldigi,gpl-3.0
1,calfzhou/dotfiles,mit
2,calfzhou/dotfiles,mit
3,HubSpot/BidHub-iOS,apache-2.0
4,HubSpot/BidHub-iOS,apache-2.0


## Outer Join

In [8]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "comments" table
table_ref = dataset_ref.table("comments")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


In [9]:
# Construct a reference to the "stories" table
table_ref = dataset_ref.table("stories")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,id,by,score,time,time_ts,title,url,text,deleted,dead,descendants,author
0,6940813,sarath237,0,1387536270,2013-12-20 10:44:30+00:00,Sheryl Brindo Hot Pics,http://www.youtube.com/watch?v=ym1cyxneB0Y,Sheryl Brindo Hot Pics,None,True,NaN,sarath237
1,6991401,123123321321,0,1388508751,2013-12-31 16:52:31+00:00,Are you people also put off by the culture of ...,,They&#x27;re pretty explicitly &#x27;startup f...,None,True,NaN,123123321321
2,1531556,ssn,0,1279617234,2010-07-20 09:13:54+00:00,New UI for Google Image Search,http://googlesystem.blogspot.com/2010/07/googl...,Again following on Bing's lead.,None,None,0.0,ssn
3,5012398,hoju,0,1357387877,2013-01-05 12:11:17+00:00,Historic website screenshots,http://webscraping.com/blog/Generate-website-s...,Python script to generate historic screenshots...,None,None,0.0,hoju
4,7214182,kogir,0,1401561740,2014-05-31 18:42:20+00:00,Placeholder,,Mind the gap.,None,None,0.0,kogir


In [10]:
query = """
        SELECT stories.id AS story_id, stories.by, stories.title, comments.ranking
        FROM `bigquery-public-data.hacker_news.stories` AS stories
        LEFT JOIN `bigquery-public-data.hacker_news.comments` AS comments
        ON stories.id = comments.parent
        WHERE EXTRACT(DATE FROM stories.time_ts) = '2012-01-01'
        """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=TWO_GB)
comment_counts = client.query(query, job_config=safe_config).to_dataframe()
comment_counts

,story_id,by,title,ranking
0,3413841,leak,Ask HN: Has the G+ HN circle changed your usag...,0.0
1,3413219,rbanffy,There’s no time like anytime,NaN
2,3412749,kaishin,2011: A year in Interface Design,NaN
3,3412710,mmichael0070,Stoner Quotes,NaN
4,3413769,lpnotes,Learning How To Code == Saving The World. Sort...,NaN
...,...,...,...,...
1097,3412570,bane,"Well, There's No Longer Any Mystery Why HP Fir...",NaN
1098,3412783,mmaltiar,Working With Spring Data JPA,NaN
1099,3412821,progga,Networking on the Network: A Guide to Professi...,NaN
1100,3412930,shipcode,Project Zero Operating System – New Kernel,NaN
